<h1>Extracting the table from Wikipedia page to Dataframe</h1>

<h4>Import the necessary packages and modules</h4>

In [1]:
import pandas as pd
import requests
! pip install BeautifulSoup4
from bs4 import BeautifulSoup
! pip install tabulate
from tabulate import tabulate
! pip install lxml

<h4>Using BeautifulSoup I have extracted the table from the specified url and saved it in the dataframe "df1" </h4>

In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df1 = pd.read_html(str(table))[0]

In [3]:
df1

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


<h4>Now, from df1 I have removed all the rows where Borough='Not assigned' and saved it in a new dataframe df</h4>

In [4]:
df = df1[~df1.Borough.str.contains("Not assigned")]

In [5]:
#reseting the previous index after removal of certain rows
df = df.reset_index(drop=True)

<h4> using isnull I am checking to see if there are any Null values in neighborhood column</h4>

In [6]:
bool_series = pd.isnull(df["Neighborhood"]) 
df[bool_series]
#from output we can observe that there are no such values

,Postal code,Borough,Neighborhood


<h4>Replacing "/" with "," in Neighborhood column</h4>

In [7]:
df['Neighborhood'] = df['Neighborhood'].str.replace('/',',')
df

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


In [8]:
#checking to see if there are any redundant Postal codes so that we can merge them
df['Postal code'].value_counts()

M4G    1
M4M    1
M1L    1
M1W    1
M1K    1
      ..
M2L    1
M6H    1
M6N    1
M3L    1
M9A    1
Name: Postal code, Length: 103, dtype: int64

In [9]:
df.shape

(103, 3)

<h4>Extract the geospatial coordinates into a dataframe</h4>

In [10]:
df2 = pd.read_csv("Geospatial_Coordinates.csv")

In [11]:
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h4>Merge the 2 dataframes to form a single dataframe with latitude and longitude values</h4>

In [13]:
df_merge_col = pd.merge(df, df2, left_on='Postal code',right_on='Postal Code')
df_merge_col

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",M7A,43.662301,-79.389494
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",M8X,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,M4Y,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,M7Y,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",M8Y,43.636258,-79.498509


<h4>drop the redundant postal code column</h4>

In [14]:
df_merge_col.drop(['Postal Code'],axis=1,inplace=True)

In [15]:
df_merge_col

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509
